In [3]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter

In [4]:
stocks = pd.read_csv('/Users/hashimkhan/Desktop/algo_trading_final/sp_500_stocks.csv')
from secret_case import API_TOKEN

In [9]:
#Function sources from: https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [20]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
dataframes = []
rows= []
dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        symbol = result.get('symbol', np.nan)
        
        year_high_percent = float(result.get('fiftyTwoWeekLowChangePercent', np.nan))
        year_low_percent = float(result.get('fiftyTwoWeekHighChangePercent', np.nan))

        year_price_return = float((year_high_percent + year_low_percent) / 2)

        regular_market_price = float(result.get('regularMarketPrice', np.nan))
        market_cap = float(result.get('marketCap', np.nan))

        rows.append([symbol, regular_market_price, year_price_return, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,110.50,-0.148367,NaN
1,AAL,12.61,-0.128348,NaN
2,AAP,54.82,-0.356030,NaN
3,AAPL,170.43,0.116156,NaN
4,ABBV,153.13,0.040090,NaN
...,...,...,...,...
499,YUM,121.98,0.012371,NaN
500,ZBH,110.76,-0.089179,NaN
501,ZBRA,225.91,-0.171387,NaN
502,ZION,33.17,0.203902,NaN
